# Optional PyTorch Matrix Factorization (MF)
This notebook mirrors `torch_mf.py` and lets you experiment interactively.
If PyTorch isn't installed or not supported for your Python version, skip the demo or install CPU-only wheels.

In [1]:
# Optional: install torch (CPU) if not present.
# You may need to restart the kernel after installation.
# %pip install --upgrade pip
# %pip install torch --index-url https://download.pytorch.org/whl/cpu

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    print(f'PyTorch {torch.__version__} loaded.')
except Exception as e:
    torch = None
    print('Torch import failed:', e)

PyTorch 2.8.0+cpu loaded.


In [2]:
from typing import List, Dict, Any

if torch is not None:
    class MF(nn.Module):
        def __init__(self, n_users: int, n_items: int, k: int = 32):
            super().__init__()
            self.U = nn.Embedding(n_users, k)
            self.V = nn.Embedding(n_items, k)
            nn.init.normal_(self.U.weight, std=0.1)
            nn.init.normal_(self.V.weight, std=0.1)

        def forward(self, ui, ii):
            u = self.U(ui)
            v = self.V(ii)
            return (u * v).sum(-1)

def train_mf(events: List[Dict[str, Any]], epochs: int = 3, k: int = 32):
    if torch is None:
        raise RuntimeError('torch not available')
    users = sorted({e['user_id'] for e in events})
    items = sorted({e['item_id'] for e in events})
    u_index = {u: i for i, u in enumerate(users)}
    i_index = {it: i for i, it in enumerate(items)}
    ui = torch.tensor([u_index[e['user_id']] for e in events], dtype=torch.long)
    ii = torch.tensor([i_index[e['item_id']] for e in events], dtype=torch.long)
    y = torch.tensor([1.0 for _ in events], dtype=torch.float32)
    model = MF(len(users), len(items), k)  # type: ignore[name-defined]
    opt = optim.Adam(model.parameters(), lr=1e-2)
    loss_fn = nn.MSELoss()
    for _ in range(epochs):
        opt.zero_grad()
        pred = model(ui, ii)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
    return u_index, i_index, model.U.weight.detach(), model.V.weight.detach()

In [3]:
# Demo: train on tiny synthetic interactions (skip if torch unavailable)
events = [
    {'user_id': 'u1', 'item_id': 'i1'},
    {'user_id': 'u1', 'item_id': 'i2'},
    {'user_id': 'u2', 'item_id': 'i2'},
    {'user_id': 'u2', 'item_id': 'i3'},
]

if torch is None:
    print('Torch not available; skipping demo.')
else:
    u_idx, i_idx, U, V = train_mf(events, epochs=5, k=8)
    print('Users index:', u_idx)
    print('Items index:', i_idx)
    print('User embedding shape:', U.shape)
    print('Item embedding shape:', V.shape)

Users index: {'u1': 0, 'u2': 1}
Items index: {'i1': 0, 'i2': 1, 'i3': 2}
User embedding shape: torch.Size([2, 8])
Item embedding shape: torch.Size([3, 8])
